In [6]:
import torch 
from torch import nn

class DenseLayer(nn.Module):
    """Fully connected linear layer."""
    
    def __init__(self, in_shape, out_shape):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(in_shape, out_shape), requires_grad=True)
    
    def forward(self, inputs):
        return torch.matmul(inputs, self.weights)

network = nn.Sequential(DenseLayer(512, 16), nn.ReLU(), DenseLayer(16, 8), nn.ReLU(), DenseLayer(8, 2))
inputs = torch.randn(8, 512)  # Batchsize 8
outputs = network(inputs)
print(outputs.size())
print(issubclass(nn.Sequential, nn.Module))

torch.Size([8, 2])
True


In [19]:
from pytorch_lightning import LightningModule

class LightningNetwork(LightningModule):
    
    def __init__(self, layer_configs=((512, 16), (16, 8), (8, 2))):
        self.save_hyperparameters()
        layers = []
        for layer_conf in layer_configs:
            layers.extend([DenseLayer(*layer_conf), nn.ReLU])
        self.model = nn.Sequential(*layers)
    
    def forward(self, inputs):
        self.model(inputs)
    
    def training_step(self, inputs, targets):
        ouputs = self(inputs)
        loss_fn = nn.CrossEntropyLoss()
        loss = loss(nn.functional.softmax(outputs, dim=1), targets)
        return loss  
    
    def configure_optimizers(self):
        return torch.optim.RMSprop(params=self.parameters(), lr=5e-5)

In [20]:
lightning_network = LightningNetwork()

AssertionError: failed to inspect the obj init